In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy

# %load_ext autoreload
# %autorelod

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import quandl  # importing the quandl for stock price
from datetime import date

token = 'aQHZz7rEw1hVor7wGXQ7'  # quandl token for python

# Set start and end dates(Date Format: yyyy/mm/dd)
start_date = "2018-01-08"
end_date = "2020-01-08"
ticker = "NSE/SBIN"  # Set the ticker

data = quandl.get(ticker, start_date=start_date,
                  end_date=end_date, authtoken=token)

data = data[['Close', 'Total Trade Quantity']]

# rename the column "Totoal trade quantity" to volume
data.rename(columns={'Total Trade Quantity': 'Volume'}, inplace=True)

data['daily_returns'] = data['Close'].pct_change()  # daily return of stock
data['daily_returns'].dropna(inplace=True)

data.tail()

,Close,Volume,daily_returns
Date,,,
2018-12-31,295.90,9526067.0,0.003731
2019-01-01,299.60,11837127.0,0.012504
2019-01-02,293.90,25559853.0,-0.019025
2019-01-03,291.10,17548347.0,-0.009527
2019-01-04,297.65,19514041.0,0.022501


In [28]:
# # Correlations
# # Create 5-day % changes of Adj_Close for the current day, and 5 days in the future
# data['5d_close_future'] = data['Close'].shift(-5)
# data['5d_close_future_pct'] = data['5d_close_future'].pct_change(5)
# data['5d_close_pct'] = data['Close'].pct_change(5)

# # Calculate the correlation matrix between the 5d close pecentage changes (current and future)
# corr = data[['5d_close_pct', '5d_close_future_pct']].corr()
# print(corr)

# # Scatter the current 5-day percent change vs the future 5-day percent change
# plt.scatter(data['5d_close_pct'], data['5d_close_future_pct'])
# plt.show()

In [4]:
#Creating the feature for the Training and test set
import talib

data['daily_returns'] = data['Close'].pct_change() # daily return of DATA
#data['daily_returns'].dropna(inplace=True)


feature_names = ['5d_close_pct']  # a list of the feature names for later

# Create moving averages and rsi for timeperiods of 14, 30, 50, and 200
for n in [14, 30, 50, 200]:

    # Create the moving average indicator and divide by Adj_Close
    data['ma' + str(n)] = talib.SMA(data['Close'].values,
                              timeperiod=n) / data['Close']
    # Create the RSI indicator
    data['rsi' + str(n)] = talib.RSI(data['Close'].values, timeperiod=n)
    
    # Add rsi and moving average to the feature name list
    feature_names = feature_names + ['ma' + str(n), 'rsi' + str(n)]

#Create features and targets
# Drop all na values
data = data.dropna()

# Create features and targets
# use feature_names for features; '5d_close_future_pct' for targets
features = data[feature_names]
targets = data['5d_close_future_pct']

# Create DataFrame from target column and feature columns
feature_and_target_cols = ['5d_close_future_pct'] + feature_names
feat_targ_df = data[feature_and_target_cols]

# Calculate correlation matrix
corr = feat_targ_df.corr()
print(corr)



KeyError: "['5d_close_pct'] not in index"

In [27]:
# #Check the correlations
# # Plot heatmap of correlation matrix
# import seaborn as sns
# sns.heatmap(corr, annot=True)
# plt.yticks(rotation=0); plt.xticks(rotation=90)  # fix ticklabel directions
# plt.tight_layout()  # fits plot area to the plot, "tightly"
# plt.show()  # show the plot
# plt.clf()  # clear the plot area

# # Create a scatter plot of the most highly correlated variable with the target
# plt.scatter(sbin['ma200'], sbin['5d_close_future_pct'])
# plt.show()

In [29]:
#Linear model #Create train and test features
# Import the statsmodels.api library with the alias sm
import statsmodels.api as sm

# Add a constant to the features
linear_features = sm.add_constant(features)

# Create a size for the training set that is 85% of the total number of samples
train_size = int(0.85 * features.shape[0])
train_features = linear_features[:train_size]
train_targets = targets[:train_size]
test_features = linear_features[train_size:]
test_targets = targets[train_size:]
print(linear_features.shape, train_features.shape, test_features.shape)



ImportError: DLL load failed: The specified procedure could not be found.